# CASO PRÁCTICO 1: OPTIMIZACIÓN DE FLUJOS EN ALMACENES CON QLEARNING

In [223]:
#INICIALIZACIÓN DE LIBRERÍAS
import numpy as np
import pandas as pd

In [224]:
#Configuración de parámetros gamma y alpha para el algoritmo de qlearning
gamma = 0.75
alpha = 0.9

### PARTE 1 - DEFINICIÓN DEL ENTORNO

In [225]:
#Definición de los estados
location_to_state = {'A':0,
                     'B':1,
                     'C':2,
                     'D':3,
                     'E':4,
                     'F':5,
                     'G':6,
                     'H':7,
                     'I':8,
                     'J':9,
                     'K':10,
                     'L':11
                    }

In [226]:
#Definición de las acciones
actions = list(range(0,12))

In [227]:
#Definición de las recompensas
R = np.array([[0,1,0,0,0,0,0,0,0,0,0,0], # A
              [1,0,1,0,0,1,0,0,0,0,0,0], # B
              [0,1,0,0,0,0,1,0,0,0,0,0], # C
              [0,0,0,0,0,0,0,1,0,0,0,0], # D
              [0,0,0,0,0,0,0,0,1,0,0,0], # E
              [0,1,0,0,0,0,0,0,0,1,0,0], # F
              [0,0,1,0,0,0,1,1,0,0,0,0], # G
              [0,0,0,1,0,0,1,0,0,0,0,1], # H
              [0,0,0,0,1,0,0,0,0,1,0,0], # I
              [0,0,0,0,0,1,0,0,1,0,1,0], # J
              [0,0,0,0,0,0,0,0,0,1,0,1], # K
              [0,0,0,0,0,0,0,1,0,0,1,0]])# L

### PARTE 2 - CONSTRUCCIÓN DE LA SOLUCIÓN DE IA CON Q-LEARNING

In [228]:
# Implementación del proceso de Q-learning
def qlearning(R_new):
    # Inicialización de los valores de Q
    Q = np.zeros(shape = (12,12))
    for i in range(1000):
        current_state = np.random.randint(0,12) # estado actual s(t)
        current_action = np.random.randint(0,12)
        while R_new[current_state][current_action] < 1:
            current_action = np.random.randint(0,12)
        next_state = current_action
        TD = R_new[current_state][current_action] + (gamma * Q[next_state][np.argmax(Q[next_state,])]) - Q[current_state][current_action]
        Q[current_state][current_action] += alpha*TD
    return Q

### PARTE 3 - PONER EL MODELO EN PRODUCCIÓN

In [230]:
state_to_location = {y:x for x,y in location_to_state.items()}

In [231]:
# Función que devuelve la ruta óptima

def route(starting_location, ending_location):
    R_new = np.copy(R)
    ending_state = location_to_state[ending_location]
    R_new[ending_state, ending_state] = 1000
    Q = qlearning(R_new) # Llamamos a la función qlearning y le pasamos R_new para calcular Q
    route = [starting_location]
    next_location = starting_location
    while next_location != ending_location:
        starting_state = location_to_state[starting_location]
        next_state = np.argmax(Q[starting_state,])
        next_location = state_to_location[next_state]
        route.append(next_location)
        starting_location = next_location
    return route

In [381]:
# Creamos la función para la ruta óptima
def optimal_route(starting_location, first_stop, ending_location):
    opt_route = route(starting_location, first_stop)[:-1] + route(first_stop, ending_location)
    return opt_route
# Imprimimos la ruta óptima
optimal_route('E', 'K','G')

['E', 'I', 'J', 'K', 'L', 'H', 'G']